In [46]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import parser.parser as parser
from lxml import etree
import re
import os
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, plugin
from rdflib.serializer import Serializer
import rdflib
from rdflib.namespace import DCTERMS, RDFS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
rootpath = '/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken/'
#ecli = 'ECLI:NL:HR:2009:BH2815'
ecli = 'ECLI:NL:XX:2006:AY0534'
element = parser.retrieve_from_any(ecli, rootpath)

In [75]:
g = Graph()
g = parser.parse_xml_element(g, element, ecli)

In [76]:
print(len(g))

4


In [78]:
for s, p, o in g:
    print(s, p, o)

http://deeplink.rechtspraak.nl/uitspraak?ECLI:NL:XX:2006:AY0534 http://purl.org/dc/terms/type http://psi.rechtspraak.nl/uitspraak
http://deeplink.rechtspraak.nl/uitspraak?ECLI:NL:XX:2006:AY0534 http://purl.org/dc/terms/hasVersion NJ 2006, 593 met annotatie van M.R. Mok
http://deeplink.rechtspraak.nl/uitspraak?ECLI:NL:XX:2006:AY0534 http://purl.org/dc/terms/date 2006-07-04
http://deeplink.rechtspraak.nl/uitspraak?ECLI:NL:XX:2006:AY0534 http://purl.org/dc/terms/hasVersion RvdW 2006, 755


In [63]:
g = Graph()
rdf = list(element.iterdescendants('{*}RDF'))[0]
nsmap = rdf.nsmap
for k, v in nsmap.items():
    g.bind(k, v)
descriptions = list(rdf.iterchildren('{*}Description'))

In [66]:
descriptions

[<Element {http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description at 0x7f3856d7b908>,
 <Element {http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description at 0x7f3856d7b9c8>]

In [68]:
ecli_node = parser.get_identifier(descriptions, ecli)
#parser.add_version(descriptions, g, ecli_node)
parser.add_reference(descriptions, g, ecli_node)

<Graph identifier=N036626edc64a4d1f8351084d4de89f59 (<class 'rdflib.graph.Graph'>)>

In [69]:
for s, p, o in g:
    print(p, o)

http://purl.org/dc/terms/title Wet op het consumentenkrediet
http://www.w3.org/2000/01/rdf-schema#label Wetsverwijzing
http://purl.org/dc/terms/reference bwb-dl1.0:v:BWB:BWBR0004815&g=2009-06-05


In [60]:
reference_list = parser.get_from_descriptions(descriptions, 'dcterms', 'references')
for reference in reference_list:
    g = add_reference(reference, g, ecli_node)

In [62]:
for s, p, o in g:
    print(s, p, o)

bwb-dl1.0:v:BWB:BWBR0004815&g=2009-06-05 http://www.w3.org/2000/01/rdf-schema#label Wetsverwijzing
bwb-dl1.0:v:BWB:BWBR0004815&g=2009-06-05 http://purl.org/dc/terms/title Wet op het consumentenkrediet
http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2009:BH2815 http://purl.org/dc/terms/reference bwb-dl1.0:v:BWB:BWBR0004815&g=2009-06-05


In [51]:
resourceIdentifier

'1.0:v:BWB:BWBR0004815&g=2009-06-05'

In [35]:
xmlstr = etree.tostring(reference, encoding=str, pretty_print=True)
print(xmlstr)

<dcterms:references xmlns:dcterms="http://purl.org/dc/terms/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:bwb="bwb-dl" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:ecli="https://e-justice.europa.eu/ecli" xmlns:tr="http://tuchtrecht.overheid.nl/" xmlns:eu="http://publications.europa.eu/celex/" xmlns:cvdr="http://decentrale.regelgeving.overheid.nl/cvdr/" xmlns:psi="http://psi.rechtspraak.nl/" rdfs:label="Wetsverwijzing" bwb:resourceIdentifier="1.0:v:BWB:BWBR0004815&amp;g=2009-06-05">Wet op het consumentenkrediet</dcterms:references>
      



In [22]:
dict(g.namespaces())

{'bwb': rdflib.term.URIRef('bwb-dl'),
 'cvdr': rdflib.term.URIRef('http://decentrale.regelgeving.overheid.nl/cvdr/'),
 'dcterms': rdflib.term.URIRef('http://purl.org/dc/terms/'),
 'ecli': rdflib.term.URIRef('https://e-justice.europa.eu/ecli'),
 'eu': rdflib.term.URIRef('http://publications.europa.eu/celex/'),
 'ns1': rdflib.term.URIRef('bwb:'),
 'psi': rdflib.term.URIRef('http://psi.rechtspraak.nl/'),
 'rdf': rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
 'rdfs': rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'),
 'tr': rdflib.term.URIRef('http://tuchtrecht.overheid.nl/'),
 'xml': rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'),
 'xsd': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')}